In [35]:
!pip install NetworkX

     ---------------------------------------- 2.0/2.0 MB 4.2 MB/s eta 0:00:00


In [2]:
import tqdm
import torch
import pandas as pd
from Bio import SeqIO
import networkx as nx

C:\Users\levi\anaconda3\envs\BigDataMahdi\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# import tarfile
  
# # open file
# file = tarfile.open('temp_outs.tar.gz')
  
# # extracting file
# file.extractall('./')
  
# file.close()

In [4]:
seq_1 = torch.load(f"temp_outs\outputs\QBZ81690.1.pt")
seq_2 = torch.load(f"temp_outs\outputs\AGO83968.2.pt")
seq_1

{'label': 'QBZ81690.1',
 'representations': {36: tensor([[ 0.0305, -0.2712, -0.0361,  ..., -0.1840, -0.0522, -0.0922],
          [ 0.1543, -0.1236, -0.0234,  ..., -0.0630, -0.2551,  0.0359],
          [ 0.0661, -0.0476, -0.0673,  ..., -0.0502, -0.2257,  0.0975],
          ...,
          [ 0.0282,  0.0571,  0.0858,  ..., -0.0320, -0.1567,  0.0178],
          [ 0.0330, -0.1428,  0.0135,  ..., -0.0338, -0.0357, -0.1182],
          [-0.1102, -0.0696, -0.1198,  ..., -0.0898, -0.1002, -0.0122]])},
 'mean_representations': {36: tensor([ 0.0314, -0.0491, -0.0131,  ..., -0.0286, -0.0542,  0.0002])}}

In [5]:
# Compute all pairwise distances
data = []
for i in range(seq_1["representations"][36].shape[0]):
    line = []
    for j in range(seq_2["representations"][36].shape[0]):
        v1 = seq_1["representations"][36][i]
        v2 = seq_2["representations"][36][j]
        line.append(round(torch.cdist(v1.unsqueeze(0), v2.unsqueeze(0)).item(), 2))
    data.append(line)
    

# Write the matrix to the excel sheet    
data = pd.DataFrame(data)
data

0     1     2     3     4     5     6     7     8     9    ...   98   \
0   4.85  7.74  7.44  7.62  7.52  7.21  7.61  7.44  7.18  7.28  ...  6.24   
1   7.78  7.29  7.96  8.19  7.94  7.76  8.02  8.16  6.97  7.92  ...  7.32   
2   7.28  7.33  7.06  7.43  6.92  6.78  7.45  7.38  7.08  6.97  ...  6.10   
3   7.18  7.67  7.11  7.23  7.23  6.96  7.19  7.27  7.15  7.04  ...  5.16   
4   6.94  7.04  6.88  7.00  6.83  6.72  7.06  6.98  6.90  6.79  ...  5.89   
..   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
94  6.73  7.26  6.89  7.00  6.54  5.83  6.99  6.91  6.42  6.58  ...  5.15   
95  7.01  7.47  7.12  7.47  6.89  6.76  7.27  7.35  6.78  6.77  ...  5.82   
96  6.74  7.25  6.88  7.10  6.58  5.84  6.94  6.98  6.63  6.58  ...  5.30   
97  7.12  7.52  7.38  7.54  7.10  6.96  7.25  7.33  6.98  7.20  ...  5.98   
98  7.21  7.60  7.45  7.50  7.18  7.12  7.46  7.37  7.17  7.23  ...  6.13   

     99    100   101   102   103   104   105   106   107  
0   6.36  6.65  6.82  6.46  6.24  6.15  6.01  6.34  6.48  
1   7.48  6.56  7.47  7.37  7.08  6.90  6.80  7.03  7.06  
2   6.50  6.66  6.32  6.13  6.07  5.90  5.94  6.17  6.29  
3   5.61  6.76  6.65  4.90  4.80  5.86  5.65  6.08  6.43  
4   6.30  6.44  6.50  5.93  5.53  5.38  5.39  5.81  6.09  
..   ...   ...   ...   ...   ...   ...   ...   ...   ...  
94  5.68  5.81  5.82  5.24  4.84  3.61  4.52  5.04  5.47  
95  6.21  6.13  6.37  5.70  5.29  5.10  4.82  5.43  5.90  
96  5.81  5.82  5.88  5.28  4.91  3.84  4.44  4.80  5.37  
97  6.23  6.38  6.48  5.85  5.49  5.29  4.96  5.08  5.76  
98  6.41  6.69  6.74  6.15  5.91  5.80  5.67  5.63  5.05  

[99 rows x 108 columns]

In [6]:
seqs = SeqIO.to_dict(SeqIO.parse("test/ab.fa", 'fasta'))

seq_ids = list(seqs.keys())
seq_ids = [seq_ids[0]] + [seq_ids[1]]
seq_ids

['QBZ81690.1', 'AGO83968.2']

In [11]:
top_n = 5
rows = pd.DataFrame({n: data.T[col].nsmallest(top_n).index.tolist() 
                  for n, col in enumerate(data.T)}).T

cols = pd.DataFrame({n: data[row].nsmallest(top_n).index.tolist() 
                  for n, row in enumerate(data)})

matches = set()
for i, n_cols in enumerate(rows.values):
    for c in n_cols:
        if i in cols.iloc[:, c].values:
            matches.add((i, int(c)))

temp_set = set()
for match in matches:
    if match[0] > 0 and match[1] > 0 and (seqs[seq_ids[0]][match[0]-1] ==  seqs[seq_ids[1]][match[1]-1]):
            temp_set.add((match[0]-1, match[1]-1))
            #if sheet[(match[0]+1, match[1]+1)].color is None:
            #    sheet[(match[0]+1, match[1]+1)].color = "#82C341"
    if match[0] < len(seqs[seq_ids[0]]) - 1  and match[1] < len(seqs[seq_ids[1]])-1 and  (seqs[seq_ids[0]][match[0]+1] ==  seqs[seq_ids[1]][match[1]+1]):
            temp_set.add((match[0]+1, match[1]+1))
            #if sheet[(match[0]+1, match[1]+1)].color is None:
            #    sheet[( match[0]+3, match[1]+3)].color = "#82C341"

matches = matches.union(temp_set)
list(matches)[0:10]

[(38, 23),
 (16, 84),
 (78, 68),
 (96, 104),
 (36, 71),
 (27, 13),
 (18, 44),
 (79, 60),
 (29, 41),
 (74, 73)]

In [12]:
def build_paths_graph(matches):
    graph = nx.DiGraph()
    dag = {}
    max_depth = max([x[0] for x in matches])
    sorted_matches = sorted(matches, key=lambda x: x[1])
    for i in range(len(sorted_matches) - 1):
        last_depth =  max_depth
        dag[sorted_matches[i]] = []
        #print(f"Comparing {sorted_matches[i]} and {sorted_matches[j]}")
        for j in range(i+1, len(sorted_matches)):
            if (sorted_matches[j][0]) < last_depth and  (sorted_matches[j][0] >  sorted_matches[i][0]):
                dag[sorted_matches[i]].append(sorted_matches[j])
                seq_1_idx, seq_2_idx = sorted_matches[j]
                graph.add_edge(sorted_matches[i], sorted_matches[j], weigth=data.iloc[seq_1_idx, seq_2_idx])
                last_depth = sorted_matches[j][0]
    return graph


graph = build_paths_graph(matches)
longest_path = nx.dag_longest_path(graph)
#len(longest_path)
longest_path[0:10]

[(0, 0),
 (9, 13),
 (10, 14),
 (12, 15),
 (13, 22),
 (14, 25),
 (26, 28),
 (29, 41),
 (32, 47),
 (37, 49)]